In [ ]:
import tensorflow_datasets as tfds


def convert_sample(sample):
    image, label = sample['image'], sample['label']  
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image, image  # Use the image itself as the target label

def map_func(sample):
    image = sample['image']
    label = sample['label']
    label = tf.one_hot(label, 2, dtype=tf.float32)
    return image, label


# Corrected file path with escaped backslashes and a missing comma
ds1, ds2, ds3 = tfds.load('patch_camelyon',
                         split=['train[:20%]', 'test[:5%]', 'validation[:5%]'],
                         data_dir=r'C:\Users\jeppe\OneDrive\Dokumenter\Privat\SDU\DataScience\Anvendtmaskinlæring\eksamen',
                         download=False,
                         shuffle_files=True)

train_dataset       = ds1.map(convert_sample).batch(32)
validation_dataset  = ds3.map(convert_sample).batch(32)
test_dataset        = ds2.map(convert_sample).batch(32)

train_dataset_cnn       = ds1.map(map_func).batch(32)
validation_dataset_cnn  = ds3.map(map_func).batch(32)
test_dataset_cnn        = ds2.map(map_func).batch(32)


In [ ]:
#bedste optimizer loop

import tensorflow.compat.v1 as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam, SGD, Adagrad, Adadelta, Adamax, Nadam, RMSprop
import matplotlib.pyplot as plt

# Your provided VAE classes (Encoder, Decoder, VAE) go here


import tensorflow.compat.v1 as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.losses import MeanSquaredError

# Define the encoder part
class Encoder(tf.keras.layers.Layer): #initialiserer encoderen, definerer lagene og præciserer hvordan vores model skal konstrueres
    def __init__(self, latent_dim): 
        super(Encoder, self).__init__()
        self.conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')
        self.maxpool1 = layers.MaxPooling2D((2, 2), padding='same')
        self.conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')
        self.maxpool2 = layers.MaxPooling2D((2, 2), padding='same')
        self.flatten = layers.Flatten()
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)

    def call(self, x): #den faktiske model der forward pass'er data og transformerer det på baggrund af hvad der er defineret i def _init_
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        mean = self.dense_mean(x)
        log_var = self.dense_log_var(x)
        return mean, log_var

# Define the decoder part
class Decoder(tf.keras.layers.Layer):
    def __init__(self, latent_dim, original_shape):
        super(Decoder, self).__init__()
        self.dense = layers.Dense(units=original_shape[0] * original_shape[1] * original_shape[2], activation=tf.nn.relu)
        self.reshape = layers.Reshape(target_shape=(original_shape[0], original_shape[1], original_shape[2]))
        self.conv2dtranspose1 = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')
        self.conv2dtranspose2 = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')
        self.conv2dtranspose3 = layers.Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')

    def call(self, z):
        z = self.dense(z)
        z = self.reshape(z)
        z = self.conv2dtranspose1(z)
        z = self.conv2dtranspose2(z)
        reconstructed = self.conv2dtranspose3(z)
        return reconstructed

# Define the Variational Autoencoder as a whole
class VAE(Model):
    def __init__(self, latent_dim, original_shape):
        super(VAE, self).__init__()
        self.encoder = Encoder(latent_dim)
        self.decoder = Decoder(latent_dim, original_shape)

    def sample(self, mean, log_var):
        batch = tf.shape(mean)[0]
        dim = tf.shape(mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return mean + tf.exp(0.5 * log_var) * epsilon

    def call(self, x):
        mean, log_var = self.encoder(x)
        z = self.sample(mean, log_var)
        reconstructed = self.decoder(z)
        return reconstructed

# Set the latent dimension and original image shape
latent_dim = 128
original_shape = (96, 96, 3)  # Assuming PCAM images are of shape 96x96x3

# List of optimizers to compare
optimizers = ['sgd', 'adam', 'rmsprop', 'adagrad', 'adamax', 'nadam']

# Plot colors for each optimizer
colors = ['b', 'g', 'r', 'c', 'm', 'y']

# Initialize subplots
plt.figure(figsize=(12, 6))

for i, optimizer in enumerate(optimizers):
    # Instantiate the VAE
    vae = VAE(latent_dim, original_shape)
    
     # Freeze the layers in the encoder
    vae.encoder.conv1.trainable = False
    vae.encoder.maxpool1.trainable = False
    vae.encoder.conv2.trainable = False
    vae.encoder.maxpool2.trainable = False
    vae.encoder.flatten.trainable = False
    vae.encoder.dense_mean.trainable = False
    vae.encoder.dense_log_var.trainable = False

    # Compile the model with the current optimizer
    vae.compile(optimizer=optimizer, loss=MeanSquaredError())

    # Train the VAE
    history = vae.fit(
        train_dataset,
        epochs=5,  # You can adjust the number of epochs
        batch_size=256,
        shuffle=True,
        validation_data=validation_dataset,
        callbacks=[TensorBoard(log_dir=f'/tmp/vae_{optimizer}'), EarlyStopping(monitor='val_loss', patience=3)]
    )

    # Access the training history
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Plot the training progress for the current optimizer
    plt.plot(loss, label=f'Training Loss ({optimizer})', linestyle='-', color=colors[i])
    plt.plot(val_loss, label=f'Validation Loss ({optimizer})', linestyle='--', color=colors[i])

# Set plot labels and legend
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

